In [ ]:
# default_exp tab_network

In [ ]:
#export
# initial code from https://github.com/dreamquark-ai/tabnet/blob/develop/pytorch_tabnet/tab_network.py
import torch
from torch.nn import Linear, BatchNorm1d, ReLU
import numpy as np
import fast_tabnet.sparsemax as sparsemax
from copy import deepcopy

def initialize_non_glu(module, input_dim, output_dim):
    gain_value = np.sqrt((input_dim+output_dim)/np.sqrt(4*input_dim))
    torch.nn.init.xavier_normal_(module.weight, gain=gain_value)
    # torch.nn.init.zeros_(module.bias)
    return


def initialize_glu(module, input_dim, output_dim):
    gain_value = np.sqrt((input_dim+output_dim)/np.sqrt(input_dim))
    torch.nn.init.xavier_normal_(module.weight, gain=gain_value)
    # torch.nn.init.zeros_(module.bias)
    return


class GBN(torch.nn.Module):
    """
        Ghost Batch Normalization
        https://arxiv.org/abs/1705.08741
    """
    def __init__(self, input_dim, virtual_batch_size=128, momentum=0.01):
        super(GBN, self).__init__()

        self.input_dim = input_dim
        self.virtual_batch_size = virtual_batch_size
        self.bn = BatchNorm1d(self.input_dim, momentum=momentum)

    def forward(self, x):
        chunks = x.chunk(x.shape[0] // self.virtual_batch_size +
                         ((x.shape[0] % self.virtual_batch_size) > 0))
        res = torch.Tensor([]).to(x.device)
        for x_ in chunks:
            y = self.bn(x_)
            res = torch.cat([res, y], dim=0)

        return res

class AttentiveTransformer(torch.nn.Module):
    def __init__(self, input_dim, output_dim, virtual_batch_size=128, momentum=0.02):
        """
        Initialize an attention transformer.

        Parameters
        ----------
        - input_dim : int
            Input size
        - output_dim : int
            Outpu_size
        - momentum : float
            Float value between 0 and 1 which will be used for momentum in batch norm
        """
        super(AttentiveTransformer, self).__init__()
        self.fc = Linear(input_dim, output_dim, bias=False)
        initialize_non_glu(self.fc, input_dim, output_dim)
        self.bn = GBN(output_dim, virtual_batch_size=virtual_batch_size,
                      momentum=momentum)

        # Sparsemax
        self.sp_max = sparsemax.Sparsemax(dim=-1)
        # Entmax
        # self.sp_max = sparsemax.Entmax15(dim=-1)

    def forward(self, priors, processed_feat):
        x = self.fc(processed_feat)
        x = self.bn(x)
        x = torch.mul(x, priors)
        x = self.sp_max(x)
        return x


class FeatTransformer(torch.nn.Module):
    def __init__(self, input_dim, output_dim, shared_blocks, n_glu,
                 virtual_batch_size=128, momentum=0.02):
        super(FeatTransformer, self).__init__()
        """
        Initialize a feature transformer.

        Parameters
        ----------
        - input_dim : int
            Input size
        - output_dim : int
            Outpu_size
        - shared_blocks : torch.nn.Module
            The shared block that should be common to every step
        - momentum : float
            Float value between 0 and 1 which will be used for momentum in batch norm
        """

        self.shared = deepcopy(shared_blocks)
        if self.shared is not None:
            for l in self.shared.glu_layers:
                l.bn = GBN(2*output_dim, virtual_batch_size=virtual_batch_size,
                           momentum=momentum)

        if self.shared is None:
            self.specifics = GLU_Block(input_dim, output_dim,
                                       n_glu=n_glu,
                                       first=True,
                                       virtual_batch_size=virtual_batch_size,
                                       momentum=momentum)
        else:
            self.specifics = GLU_Block(output_dim, output_dim,
                                       n_glu=n_glu,
                                       virtual_batch_size=virtual_batch_size,
                                       momentum=momentum)

    def forward(self, x):
        if self.shared is not None:
            x = self.shared(x)
        x = self.specifics(x)
        return x


class GLU_Block(torch.nn.Module):
    """
        Independant GLU block, specific to each step
    """
    def __init__(self, input_dim, output_dim, n_glu=2, first=False,
                 virtual_batch_size=128, momentum=0.02):
        super(GLU_Block, self).__init__()
        self.first = first
        self.n_glu = n_glu
        self.glu_layers = torch.nn.ModuleList()
        self.scale = torch.sqrt(torch.FloatTensor([0.5]))
        for glu_id in range(self.n_glu):
            if glu_id == 0:
                self.glu_layers.append(GLU_Layer(input_dim, output_dim,
                                                 virtual_batch_size=virtual_batch_size,
                                                 momentum=momentum))
            else:
                self.glu_layers.append(GLU_Layer(output_dim, output_dim,
                                                 virtual_batch_size=virtual_batch_size,
                                                 momentum=momentum))

    def forward(self, x):
        self.scale = self.scale.to(x.device)
        if self.first:  # the first layer of the block has no scale multiplication
            x = self.glu_layers[0](x)
            layers_left = range(1, self.n_glu)
        else:
            layers_left = range(self.n_glu)

        for glu_id in layers_left:
            x = torch.add(x, self.glu_layers[glu_id](x))
            x = x*self.scale
        return x


class GLU_Layer(torch.nn.Module):
    def __init__(self, input_dim, output_dim,
                 virtual_batch_size=128, momentum=0.02):
        super(GLU_Layer, self).__init__()

        self.output_dim = output_dim
        self.fc = Linear(input_dim, 2*output_dim, bias=False)
        initialize_glu(self.fc, input_dim, 2*output_dim)

        self.bn = GBN(2*output_dim, virtual_batch_size=virtual_batch_size,
                      momentum=momentum)

    def forward(self, x):
        x = self.fc(x)
        x = self.bn(x)
        out = torch.mul(x[:, :self.output_dim], torch.sigmoid(x[:, self.output_dim:]))
        return out
